First we start with importing needed libraries.


In [4]:
print("is this working")

is this working


In [5]:
!pip install -U scikit-learn


You should consider upgrading via the 'c:\users\user\appdata\local\programs\python\python37-32\python.exe -m pip install --upgrade pip' command.


In [6]:
!pip install pandas

You should consider upgrading via the 'c:\users\user\appdata\local\programs\python\python37-32\python.exe -m pip install --upgrade pip' command.


In [7]:
!pip install numpy 

You should consider upgrading via the 'c:\users\user\appdata\local\programs\python\python37-32\python.exe -m pip install --upgrade pip' command.


In [8]:
import numpy as np
import pandas as pd 


In [9]:
np.__version__

'1.19.0'

In [10]:
pd.__version__


'1.0.5'

In [11]:
project_path= r"E:\learning\project"
lfw_path=r"E:\learning\project\lfw"


we need a function to fetch features file from drive 
and convert it to proper ndarray format ==>


In [12]:
def get_feature(fileName,index):
    formatted_index="0000"
    if(int(index)<10):
        formatted_index="000"+str(index)
    elif(int(index)<100):
        formatted_index="00"+str(index)
    else:
        formatted_index="0"+str(index)
    abs_path=lfw_path+"\\" +fileName+"\\"+fileName+"_"+str(formatted_index)+".txt"
    file_string=open(abs_path).readlines()
    file_string=str(file_string).replace(r"\n', '  "," ")
    file_string=file_string.replace("[","")
    file_string=file_string.replace("]","")
    file_string=file_string.replace(",","")
    file_string=file_string.replace("'","")
    arr=file_string.split()
    nd_arr=np.zeros((1,512))
    for i in range(0,512):
        try:
            nd_arr[0][i]=np.float32(arr[i])
        except:
            print("this is err ",fileName,index)
            print(arr)
            break
    # nd_arr=np.fromfile(abs_path,dtype=np.float32,sep=" ")
    return nd_arr


In [13]:
print(get_feature("Aaron_Guiel",1).shape)

(1, 512)


In [14]:
#opening pair dev train 
pair_train_file=open(project_path+"\\pairsDevTrain.txt")
pair_train_list=pair_train_file.readlines()
number_of_p_classes=pair_train_list.pop(0)
print(number_of_p_classes)
pair_test_file=open(project_path+"\\pairsDevTest.txt")
pair_test_list=pair_test_file.readlines()
number_of_p_classes_test=pair_test_list.pop(0)
print(number_of_p_classes_test)


1100

500



In [15]:
pair_train_list[5]


'Abdullah\t2\t4\n'

we need a function to extract data from pardevTrain.txt and create train or test set
based on it .

In [16]:
def make_train_set(pairTrainList,numberOfClasses):
    x=np.zeros((numberOfClasses*2,1024),dtype=np.float32)
    print(x.shape)
    y=np.zeros(numberOfClasses*2,dtype=int)
    counter=-1
    for rawStr in pairTrainList :
        counter+=1
        #print(type(rawStr))
        rawStr=rawStr.replace("\n","\t")
        splitted=rawStr.split("\t")
        #print(splitted)
        if(counter<numberOfClasses):
            feature_first=get_feature(splitted[0],int(splitted[1]))
            feature_second=get_feature(splitted[0],int(splitted[2]))
            concatted=np.concatenate((feature_first,feature_second),axis=1)
            # resized=np.resize(concatted,(4032))
            x[counter]=concatted
            y[counter]=1 #positive class
        else:
            feature_first=get_feature(splitted[0],int(splitted[1]))
            feature_second=get_feature(splitted[2],int(splitted[3]))
            concatted=np.concatenate((feature_first,feature_second),axis=1)
            # resized=np.resize(concatted,(4032))
            x[counter]=concatted
            y[counter]=0 #negative class
        
    print("x set shape is : ",x.shape)
    print("y set shape is : ",y.shape)
           
        
    return x,y

In [17]:
x_train,y_train=make_train_set(pair_train_list,int(number_of_p_classes))
x_train[2000]

(2200, 1024)
x set shape is :  (2200, 1024)
y set shape is :  (2200,)


array([0.        , 1.2414856 , 2.122094  , ..., 0.        , 3.203059  ,
       0.23443337], dtype=float32)

In [18]:
x_test,y_test=make_train_set(pair_test_list,int(number_of_p_classes_test))
x_test[100]

(1000, 1024)
x set shape is :  (1000, 1024)
y set shape is :  (1000,)


array([0.03927608, 3.7009673 , 6.7248387 , ..., 0.        , 1.9627694 ,
       4.1425214 ], dtype=float32)

now our data sets are ready.
in this section we will add 4 different classifiers such as :
1.DT 2.SVM 3.Kneighbours 4.logisticRegression

and for start we use Bucket of models method to decide which of these models
can predict better.


In [19]:
#importing needed libraries
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

In [20]:
lr_clf = LogisticRegression(random_state=0,max_iter=1000,solver='lbfgs').fit(x_train, y_train)

c:\users\user\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [21]:
knn_clf = KNeighborsClassifier(n_neighbors=3).fit(x_train,y_train)

In [26]:
dt_clf = DecisionTreeClassifier(random_state=0).fit(x_train,y_train)

In [27]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
svm_clf = make_pipeline(StandardScaler(), SVC(gamma='auto')).fit(x_train,y_train)

In [28]:
# now lets test our models on a  same feature data
knn_pred=knn_clf.predict(x_test)
print("knn classifier prediction : ",knn_pred )
print("actual value: ", y_test)


knn classifier prediction :  [1 1 1 0 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 0 1 1 0 1
 0 1 1 0 1 0 1 1 1 1 1 1 1 1 0 1 0 1 1 1 0 0 1 1 1 1 0 0 1 0 1 1 1 1 1 1 1
 1 1 0 1 0 1 1 0 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1
 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 0 1
 1 1 1 0 0 0 1 1 1 1 1 1 1 0 1 1 1 0 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 0 0 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 0 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 0 1 1 1 1 1 0 1 0 1 1 1 1 0 0 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 0 1 1 1 1 0 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 0 0 0 1 0 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 0 0 1 1 1 1 1 0 1 1
 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 0 1 1 1 1 1 1 1 1 

In [29]:
dt_pred=dt_clf.predict(x_test)
print("decision tree classifier prediction : ", dt_pred)
print("actual value: ", y_test)


decision tree classifier prediction :  [1 0 0 1 0 0 0 0 0 0 1 0 1 0 1 1 1 1 0 0 1 1 1 1 1 1 0 0 1 0 1 1 0 1 1 0 0
 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 0 0 0 1 0 0 1 1 1 0 1 1 1 1 0 0 0 1 0 0 1
 0 1 1 1 1 1 1 1 1 1 0 0 0 1 0 0 0 1 1 1 0 1 1 1 1 1 1 0 1 0 1 0 1 0 0 1 0
 1 1 0 0 0 1 1 1 0 1 0 1 1 1 0 0 1 0 0 1 1 1 0 0 1 0 1 1 1 1 1 1 1 1 1 1 0
 0 0 1 0 1 1 0 0 0 0 1 0 0 1 0 1 0 1 1 1 1 1 1 1 1 1 1 0 1 0 1 0 1 1 0 1 1
 1 0 0 1 1 1 0 0 0 1 0 0 0 1 1 0 1 0 1 0 1 0 0 0 1 1 0 0 0 0 0 1 0 1 0 0 1
 1 0 0 0 0 1 1 1 1 1 1 0 0 1 1 1 1 1 0 0 0 1 1 1 1 1 1 1 0 1 1 1 1 1 0 0 0
 0 1 0 0 0 1 0 0 0 1 1 0 1 0 0 1 1 0 0 0 0 1 1 1 1 0 0 1 1 1 1 1 0 1 1 0 1
 1 1 1 0 1 1 1 1 1 1 1 1 0 1 0 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 1 1
 0 1 0 1 1 1 1 1 1 0 0 0 1 1 1 1 0 0 1 0 1 1 0 0 1 0 0 1 1 1 1 1 1 0 0 1 0
 0 0 0 0 1 0 1 0 1 0 0 1 1 1 0 1 1 1 0 0 1 0 0 1 1 1 1 0 1 0 0 0 1 0 1 1 0
 1 1 0 1 0 1 1 0 1 1 1 1 1 1 0 1 1 0 1 1 1 0 1 1 1 1 1 0 0 1 0 1 1 0 0 0 0
 0 1 0 0 0 1 0 1 1 1 0 1 0 0 0 1 1 0 1 0 1 0 0 0 1 1 1 1 1 0 

In [30]:
svm_pred=svm_clf.predict(x_test)
print("svm classifier prediction : ",svm_pred )
print("actual value: ", y_test)


svm classifier prediction :  [1 1 1 1 0 0 1 1 1 0 1 0 0 1 0 1 1 1 1 1 0 1 0 1 0 0 1 1 1 0 1 1 1 1 1 1 0
 0 1 1 0 1 0 1 1 1 0 1 1 1 1 1 1 0 1 1 1 0 1 1 0 1 1 0 1 1 0 1 1 1 1 1 1 1
 1 1 1 0 0 1 0 1 1 0 0 0 0 0 1 1 1 1 0 1 0 1 0 1 1 1 1 1 0 0 1 0 1 1 1 1 1
 1 0 1 1 1 0 1 1 1 0 1 0 1 0 1 1 1 1 0 0 0 0 1 1 1 1 1 0 0 1 1 0 1 0 0 0 1
 1 0 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1
 1 1 1 0 0 0 1 1 1 1 0 1 0 1 1 1 1 1 0 1 0 0 1 0 0 0 0 1 0 1 0 0 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 0 1 1 1 1
 1 0 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 0 0 0 1 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 0 0 1 1
 1 1 1 1 1 1 1 0 1 1 1 1 1 0 0 1 1 0 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1
 1 0 0 1 1 0 0 1 0 1 1 1 1 1 1 1 1 1 0 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 0 1 0 0 0 1 1 0 1 1 0 1 1 0 1 1 0 1 1 1 1 0 1 1 1 1 1 0 1 1
 1 0 0 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 0 1 0 1 1 1 0 

lets measure the accuracy score of classifier above :


In [31]:
from sklearn.metrics import accuracy_score
acc_svm=accuracy_score(svm_pred,y_test)
acc_dt=accuracy_score(dt_pred,y_test)
acc_knn=accuracy_score(knn_pred,y_test)
print("accuracies : svm = %s  dt= %s knn=%s ",acc_svm,acc_dt,acc_knn)


accuracies : svm = %s  dt= %s knn=%s  0.748 0.571 0.678


since our logistic regression didnt learn our data set we come up with another 
classifier
here we use mlp :      


In [32]:
from sklearn.neural_network import MLPClassifier
mlp_clf = MLPClassifier(random_state=1, max_iter=300).fit(x_train, y_train)

In [33]:
mlp_pred=mlp_clf.predict(x_test)
acc_mlp=accuracy_score(mlp_pred,y_test)
print(f"accuracies : svm = {acc_svm} dt= {acc_dt} knn={acc_knn} mlp={acc_mlp}")



accuracies : svm = 0.748 dt= 0.571 knn=0.678 mlp=0.766


Simplest and most basic ensemble method is bucket of models.
we trained 4 models and each had its own accuracy.(ps. I didnt use 
validation set on x_train and splitting data set to two part may differ the result)
we can do simple math operation to the obtained resutls :
1- maximum : which is 76% so we give 100% weight to mlp and 0% to other classifiers
2- average : average of all of our models is equal to 69% . we weight our models
based on their accuracy affecting final result :
mlp = 76/400 svm =75/400 knn=68/400 dt=57/400


for next method, we will use bootstrap aggragation(bagging) :
 sklearn.ensemble.BaggingClassifier have several parameters which I will discuss and 
 use few of them. base_estimator is a classifier that this algorithm is gonna use.
 since we are using more than 10 estimator we dont want our estimator to be 
 precise and exact. moreover, we want it to be weak in order to avoid overfitting in our data.
 as a conclusion, I use 20 decision tree and check the performance.
 as you know we have 2200 sample, if we divide them to 20 bootstrap 
 we will get 110 sample in each bootstrap. However, repetitive samples may occur.

In [197]:
from sklearn.ensemble import BaggingClassifier
bagging_clf = BaggingClassifier(base_estimator=DecisionTreeClassifier(random_state=0)
                        ,n_estimators=10000,max_samples=110,max_features=100, random_state=0).fit(x_train, y_train)



In [198]:
bagging_pred=bagging_clf.predict(x_test)
acc_bagging=accuracy_score(bagging_pred,y_test)
print(acc_bagging)


0.624


I tried using 20 DT but the result was not satisfying, after that I used 1000
DT estimator and I got 58% accuracy. I increased the max_faetures parameter to
512(all the features) but the model got over fitted and I got 53% accuracy.
Finally I trained our bagging model with 10000 estimator and only 110 samples and 100 features
and the improvement in results was beyond expectations. Having consider that in
previous method our DT gave us only 57. but with bagging and using 10K of Decision trees 
I reached the accuracy of 62.5% which can be get better if we play with params.


Now I wanna use the mlp estimator with bagging method to see any differences.


In [36]:
#unfortunately kernel crashed and I have to run cells again !!!
bagging_clf_mlp=BaggingClassifier(base_estimator= MLPClassifier(random_state=1, max_iter=300)
                        ,n_estimators=100,max_samples=110,max_features=100, random_state=0).fit(x_train, y_train)


In [37]:
bagging_mlp_pred=bagging_clf_mlp.predict(x_test)
acc_bagging_mlp=accuracy_score(bagging_mlp_pred,y_test)
print(acc_bagging_mlp)

0.605


In [38]:
#lets increase the number of estimators
bagging_clf_mlp=BaggingClassifier(base_estimator= MLPClassifier(random_state=1, max_iter=300)
                        ,n_estimators=1000,max_samples=110,max_features=100, random_state=0).fit(x_train, y_train)
bagging_mlp_pred=bagging_clf_mlp.predict(x_test)
acc_bagging_mlp=accuracy_score(bagging_mlp_pred,y_test)
print(acc_bagging_mlp)

c:\users\user\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
c:\users\user\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
c:\users\user\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
c:\users\user\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) r

0.627


62% is not precise so we change the paramteres


In [41]:
bagging_clf_mlp=BaggingClassifier(base_estimator= MLPClassifier(random_state=1, max_iter=500)
                        ,n_estimators=1000,max_samples=10,max_features=10, random_state=0).fit(x_train, y_train)
bagging_mlp_pred=bagging_clf_mlp.predict(x_test)
acc_bagging_mlp=accuracy_score(bagging_mlp_pred,y_test)
print(acc_bagging_mlp)


c:\users\user\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
c:\users\user\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
c:\users\user\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


0.533


Still no significant improvement in accuract...
lets go to the next method which is Boosting :
here we use adaboosting classifier with DT base estimator and 
maximum depth of 1, in this part we use 100,1000,10000 estimator as input parameters.


In [42]:
from sklearn.ensemble import AdaBoostClassifier
adaboost_clf = AdaBoostClassifier(n_estimators=100, random_state=0).fit(x_train,y_train)


In [43]:
adaboost_pred=adaboost_clf.predict(x_test)
acc_adaboost=accuracy_score(adaboost_pred,y_test)
print(acc_adaboost)

0.517


In [44]:
#increase to 1000
adaboost_clf = AdaBoostClassifier(n_estimators=1000, random_state=0).fit(x_train,y_train)
adaboost_pred=adaboost_clf.predict(x_test)
acc_adaboost=accuracy_score(adaboost_pred,y_test)
print(acc_adaboost)

0.522


In [45]:
#increase estimators to 10k
adaboost_clf = AdaBoostClassifier(n_estimators=10000, random_state=0).fit(x_train,y_train)
adaboost_pred=adaboost_clf.predict(x_test)
acc_adaboost=accuracy_score(adaboost_pred,y_test)
print(acc_adaboost)

0.524


for the last part I will use random forest method.

In [46]:
from sklearn.ensemble import RandomForestClassifier
random_forest_clf = RandomForestClassifier(n_estimators=100).fit(x_train,y_train)

In [47]:
random_forest_pred=random_forest_clf.predict(x_test)
acc_random_forest=accuracy_score(random_forest_pred,y_test)
print(acc_random_forest)


0.65


In [48]:
#increasing number of estimators to 1k
random_forest_clf = RandomForestClassifier(n_estimators=1000).fit(x_train,y_train)
random_forest_pred=random_forest_clf.predict(x_test)
acc_random_forest=accuracy_score(random_forest_pred,y_test)
print(acc_random_forest)

0.714


In [49]:
#increase number of estimators to 10k
random_forest_clf = RandomForestClassifier(n_estimators=10000).fit(x_train,y_train)
random_forest_pred=random_forest_clf.predict(x_test)
acc_random_forest=accuracy_score(random_forest_pred,y_test)
print(acc_random_forest)



0.731


For the conclusion, using random forest was so far the besti choice for the following 
reasons :
1- great accuracy on test set.
2- minimizing the overfitting.
3- faster than other methods when using DT as base estimator 

we can also reach higher accuracy just by adjusting DTs configurations .
in this project I used base models as simple as I can so comparison is easier.
